# Assignment 3

## Instructions - Read this first!

This is an individual homework assignment. This means that:

- You may discuss the problems in this assignment with other students in this course and your instructor/TA, but YOUR WORK MUST BE YOUR OWN.
- Do not show other students code or your own work on this assignment.
- You may consult external references, but not actively receive help from individuals not involved in this course.
- Cite all references outside of the course you used, including conversations with other students which were helpful. (This helps us give credit where it is due!). All references must use a commonly accepted reference format, for example, APA or IEEE (or another citation style of your choice).

If any of these rules seem ambiguous, please check with with your instructor for help interpreting them.

We suggest completing this assignment using the provided notebook. Each question should be answered using a SQL query (or combination or SQL queries) unless the text indicates that you may (or should) do something else. You may submit your queries embedded in Python, using SQLAlchemy or the MySQL Connector, or as plain text in Markdown.

## When you submit your work

Your submission will be graded manually. To ensure that everything goes smoothly, please follow these instructions to prepare your notebook for submission to the D2L Dropbox for Assignment 3:

- Please remove any print statments used to test your work (you can comment them out)
- Please provide your solutions where asked; please do not alter any other parts of this notebook.
- If you need to add cells to test your code please move them to the end of the notebook before submission- or you may included your commented out answers and tests in the cells provided

## Introduction

In this assignment, you will continue to practice and extend your SQL skills, and contemplate the use of MongoDB.

We will be using two datasets from the City of Calgary's Open Data Portal. 

 * `School_Enrolment_Data_20240324.csv` lists annual student enrolment from ECS to Grade 12 throughout Calgary. You can find out more about this dataset from the [City of Calgary's Open Data Portal](https://data.calgary.ca/Demographics/School-Enrolment-Data/9qye-mibh/about_data).
 * `Schools_20240324.csv` lists location and information of schools and post-secondary institutions including school name, address, phone number and grade level. . You can find out more about this dataset from the [City of Calgary's Open Data Portal](https://data.calgary.ca/Services-and-Amenities/Schools/fd9t-tdn2/about_data)

Both datasets are licensed under the [Open Government License - Calgary](https://data.calgary.ca/stories/s/Open-Calgary-Terms-of-Use/u45n-7awa).

## Data cleaning and import

First, import the two CSVs into your own database. You may use what is available to you on `datasciencedb`. You may also create indexes and define keys if appropriate for the column(s) of your choice.

In the section below, you have the option to discuss any data cleaning and wrangling steps performed during this process. This is not a requirement and will not be assessed directly for grading; however, this may help to clarify to your reader exactly what was done, to make your work below more understandable.

In [97]:
import pandas as pd
import sqlalchemy as sq

# read in your CSV as a dataframe
scenr = pd.read_csv("School_Enrolment_A3.csv")
schools = pd.read_csv("Schools_A3.csv")

In [98]:
print("Length of School_enrolment table is {0} rows".format(len(scenr)))
print("Length of Schools table is {0} rows".format(len(schools)))

Length of School_enrolment table is 3830 rows
Length of Schools table is 489 rows


In [99]:
schools_unique = schools.nunique()
schools_unique[schools_unique == len(schools)]

GLOBAL_ID    489
dtype: int64

The output suggests that there is only one column (GLOBAL_ID) in the Schools DataFrame where every value is unique across the entire column.

In [100]:
scenr_unique = scenr.nunique()
scenr_unique[scenr_unique == len(scenr)]

Series([], dtype: int64)

The output suggests that there are no columns in the Schools DataFrame where all the values are unique across the entire column.

In [101]:
# connect to your database; include a cell at the bottom of this notebook to dispose of your engine object
user = 'camila_delgado'
password = 'y8fzy2Ehmn3Xh'
host = 'datasciencedb.ucalgary.ca'
database = 'camila_delgado'

engine = sq.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.format(user, password, host, database))
# write your dataframe into a table
scenr.to_sql(name='scenr', con=engine, if_exists='replace')
schools.to_sql(name='schools', con=engine, if_exists='replace')

489

## Part A: Warm-up Questions (10 marks)

Answer the questions below, including any queries you used where necessary. Not all questions will necessarily require a SQL query for a correct response. You may wish to use as a source the references which are already provided as part of this notebook.

First, let's look at the School Enrolment Data.

1. How many schools are included in this file? **(1 mark)**


2.  While not all columns are equally interesting, name columns which could be used as a primary key for this dataset. **(2 marks)**

 
3. How many students participated in Calgary Home Education in 2018-2019 compared to 2020-2021? **(2 marks)**

In [102]:
#How many schools are included in this file?
query0 = '''
SELECT COUNT(DISTINCT `School Code`) AS num_of_schools FROM scenr;
'''

# Execute the query and read the result into a DataFrame
result_df = pd.read_sql_query(query0, engine)

# Print the result
print("Number of unique schools:", result_df['num_of_schools'][0])

Number of unique schools: 421


In [115]:
# While not all columns are equally interesting, name columns which could be used as a primary key for this dataset. (2 marks)
def assess_uniqueness(df):
    uniqueness_results = {}
    for column in df.columns:
        unique_values = df[column].unique()
        num_unique_values = len(unique_values)
        total_values = df[column].notna().sum()
        uniqueness_results[column] = {
            'num_unique_values': num_unique_values,
            'total_values': total_values,
            'uniqueness_ratio': num_unique_values / total_values if total_values > 0 else 0
        }
    return uniqueness_results

# Assess uniqueness of values in each column for 'scenr' DataFrame
uniqueness_results = assess_uniqueness(scenr)

# Display results
for column, result in uniqueness_results.items():
    print(f"Column: {column}")
    print(f"Number of unique values: {result['num_unique_values']}")
    print(f"Total non-null values: {result['total_values']}")
    print(f"Uniqueness ratio: {result['uniqueness_ratio']}\n")


Column: School Year
Number of unique values: 10
Total non-null values: 3830
Uniqueness ratio: 0.0026109660574412533

Column: School Authority Category
Number of unique values: 4
Total non-null values: 3830
Uniqueness ratio: 0.0010443864229765013

Column: School Authority Name
Number of unique values: 39
Total non-null values: 3830
Uniqueness ratio: 0.010182767624020888

Column: School Authority Code
Number of unique values: 35
Total non-null values: 3830
Uniqueness ratio: 0.009138381201044387

Column: School Name
Number of unique values: 439
Total non-null values: 3830
Uniqueness ratio: 0.11462140992167102

Column: School Code
Number of unique values: 421
Total non-null values: 3830
Uniqueness ratio: 0.10992167101827677

Column: ECS
Number of unique values: 158
Total non-null values: 2593
Uniqueness ratio: 0.060933281912842266

Column: Grade 1
Number of unique values: 165
Total non-null values: 2580
Uniqueness ratio: 0.06395348837209303

Column: Grade 2
Number of unique values: 161
Tot

## Considering the criteria and the provided uniqueness ratios, the 'School Code' or 'School Name' column could be strong candidates for the primary key. Both columns have relatively high uniqueness ratios and represent identifiers for individual schools.

## How many students participated in Calgary Home Education in 2018-2019 compared to 2020-2021? (2 marks)

In [119]:
query0 = '''

SELECT
    SUM(CASE WHEN `School Year` = '2020-2021' THEN Total ELSE 0 END) AS Total_Students_2020_2021,
    SUM(CASE WHEN `School Year` = '2018_2019' THEN Total ELSE 0 END) AS Total_Students_2018_2019,
    SUM(CASE WHEN `School Year` = '2020-2021' THEN Total ELSE 0 END) -
    SUM(CASE WHEN `School Year` = '2018_2019' THEN Total ELSE 0 END) AS Enrollment_Difference
FROM
    scenr
WHERE
    `School Name` = 'Calgary Home Education';


    '''
result_df0 = pd.read_sql_query(query0, engine)
result_df0

,Total_Students_2020_2021,Total_Students_2018_2019,Enrollment_Difference
0,420.0,197.0,223.0


Next, let's look at the School Location table. 

4. Are there any schools without phone numbers listed? **(1 mark)**


5. Which columns would be suitable primary keys for this table? **(1 mark)**


6. What forward sortation area(s) have the most schools? **(3 marks)**

In [120]:
#Are there any schools without phone numbers listed? (1 mark)

query1 = '''
SELECT NAME
FROM schools
WHERE PHONE_NO IS NULL OR PHONE_NO = '';
'''

# Execute the query and read the result into a DataFrame
result_df1 = pd.read_sql_query(query1, engine)
print(result_df1)

                                               NAME
0                             McKenzie Towne School
1                                 Bridlewood School
2                                   Somerset School
3                         Robert Thirsk High School
4                              Vista Virtual School
5  Filipino Language and Cultural School of Calgary
6                                  Evergreen School
7                        North Middle School Campus
8                       Northeast Elementary Campus
9                         Westmount Mid/High School


## Which columns would be suitable primary keys for this table? (1 mark)

GLOBAL_ID: This column appears to contain unique identifiers for each school.
NAME: The school names are likely to be unique within the dataset.
ADDRESS_AB and POSTAL_COD: Combining the address and postal code may also result in unique identifiers for each school.

## What forward sortation area(s) have the most schools? (3 marks)

In [121]:
query2 = '''
SELECT LEFT(POSTAL_COD, 3) AS Forward_Sortation_Area, COUNT(DISTINCT NAME) AS Num_Schools
FROM schools
GROUP BY Forward_Sortation_Area
ORDER BY Num_Schools DESC
LIMIT 1;
'''

result_df2 = pd.read_sql_query(query2, engine)
print(result_df2)

  Forward_Sortation_Area  Num_Schools
0                    T2A           30


## Part B: SQL with multiple tables (15 marks)

For each school, provide the name, postal code, school authority category, and school authority name. How many schools are listed if you use an inner join as opposed to a a left or right (outer) join? What explains the discrepancy? **(4 marks)**

List all schools not listed as private schools with their name and address and which did not have Grade 5 students in 2019-2020 **(1 mark)**

Are there any schools that have grades listed as "Unknown" that you could update based on the enrolment table? Get the names of these schools. Which ones could you not update with the enrolment table?  **(4 marks)**

Where there is ECS enrolment at a private school, include the address, school name, and whether the school is an elementary school or secondary school. **(2 marks)**

Report the name and addresses of the schools that have a ECS program and enrolment number for each year from 2013-2022. What is the breakdown of school types (grades in the School dataset) that have ECS enrolment in the most recent year of the enrolment dataset? **(4 marks)**

##For each school, provide the name, postal code, school authority category, and school authority name. How many schools are listed if you use an inner join as opposed to a a left or right (outer) join? What explains the discrepancy? (4 marks)

In [122]:
#INNER JOIN --> 3446 Schools are listed

query3 = '''

SELECT s.NAME, s.POSTAL_COD, sc.`School Authority Category`, sc.`School Authority Name`
FROM schools s
INNER JOIN scenr sc ON s.NAME = sc.`School Name`;

'''
result_df3 = pd.read_sql_query(query3, engine)
result_df3

,NAME,POSTAL_COD,School Authority Category,School Authority Name
0,Midsun School,T2X3R5,Public,Calgary School District No. 19
1,The Hamptons School,T3A6G2,Public,Calgary School District No. 19
2,Hidden Valley School,T3A6J2,Public,Calgary School District No. 19
3,Crossing Park School,T3J4W8,Public,Calgary School District No. 19
4,Battalion Park School,T3H4S2,Public,Calgary School District No. 19
...,...,...,...,...
3441,Battalion Park School,T3H4S2,Public,The Calgary School Division
3442,St. Anne Academic Centre,T2G1N2,Separate,The Calgary Roman Catholic Separate School Div...
3443,Peter Lougheed School,T3J5J1,Public,The Calgary School Division
3444,Battalion Park School,T3H4S2,Public,The Calgary School Division


In [123]:
#LEFT OUTER JOIN --> 3552 Schools are listed

query4 = '''
SELECT s.NAME, s.POSTAL_COD, sc.`School Authority Category`, sc.`School Authority Name`
FROM schools s
LEFT JOIN scenr sc ON s.NAME = sc.`School Name`;
'''
result_df4 = pd.read_sql_query(query4, engine)
result_df4

,NAME,POSTAL_COD,School Authority Category,School Authority Name
0,Midsun School,T2X3R5,Public,Calgary School District No. 19
1,The Hamptons School,T3A6G2,Public,Calgary School District No. 19
2,Hidden Valley School,T3A6J2,Public,Calgary School District No. 19
3,Crossing Park School,T3J4W8,Public,Calgary School District No. 19
4,Battalion Park School,T3H4S2,Public,Calgary School District No. 19
...,...,...,...,...
3547,North Trail High School,T3K2S3,None,None
3548,Rundle College Junior Senior High,T3H3W5,None,None
3549,Calgary Arts Academy Rosscarrock Middle School,T3C1W7,None,None
3550,Canadian Montessori School,T2V0P6,None,None


### The discrepancy arises from the handling of unmatched rows: Inner join focuses on the intersection of rows between the tables, excluding unmatched rows. Left (or right) outer join includes all rows from one specified table (the "left" or "right" table) while still including matching rows from the other table, filling in NULL values for unmatched rows.

## List all schools not listed as private schools with their name and address and which did not have Grade 5 students in 2019-2020 (1 mark)

In [124]:
query5 = '''
SELECT s.NAME AS School_Name,
       s.ADDRESS_AB AS School_Address
FROM schools s
LEFT JOIN scenr sc ON s.NAME = sc.`School Name` AND sc.`School Year` IN (2019, 2020)
WHERE sc.`School Authority Category` IS NOT NULL
  AND sc.`School Authority Category` <> 'Private'
  AND (sc.`Grade 5` IS NULL OR sc.`Grade 5` = 0);
'''

result_df5 = pd.read_sql_query(query5, engine)
result_df5

,School_Name,School_Address
0,Alternative High School,5003 20 St SW
1,James Fowler High School,4004 4 St NW
2,Bowness High School,4627 77 St NW
3,Ian Bazalgette School,3909 26 Ave SE
4,New Brighton School,30 New Brighton Dr SE
...,...,...
185,Annie Gale School,577 Whiteridge Way NE
186,Royal Oak School,9100 Royal Birch Blvd NW
187,Lord Beaverbrook High School,9019 Fairmount Dr SE
188,Somerset School,150 Somerset Manor SW


In [125]:
query6 = '''
SELECT s.NAME, s.ADDRESS_AB, sc.`School Authority Category`
FROM schools s
LEFT JOIN scenr sc ON s.NAME = sc.`School Name` AND sc.`School Year` = '2019_2020';


'''
result_df6 = pd.read_sql_query(query5, engine)
result_df6

,School_Name,School_Address
0,Alternative High School,5003 20 St SW
1,James Fowler High School,4004 4 St NW
2,Bowness High School,4627 77 St NW
3,Ian Bazalgette School,3909 26 Ave SE
4,New Brighton School,30 New Brighton Dr SE
...,...,...
185,Annie Gale School,577 Whiteridge Way NE
186,Royal Oak School,9100 Royal Birch Blvd NW
187,Lord Beaverbrook High School,9019 Fairmount Dr SE
188,Somerset School,150 Somerset Manor SW


In [126]:
query7 = '''
SELECT DISTINCT s.NAME, s.ADDRESS_AB, sc.`School Authority Category`
FROM schools s
LEFT JOIN scenr sc ON s.NAME = sc.`School Name`;

'''
result_df7 = pd.read_sql_query(query7, engine)
result_df5

,School_Name,School_Address
0,Alternative High School,5003 20 St SW
1,James Fowler High School,4004 4 St NW
2,Bowness High School,4627 77 St NW
3,Ian Bazalgette School,3909 26 Ave SE
4,New Brighton School,30 New Brighton Dr SE
...,...,...
185,Annie Gale School,577 Whiteridge Way NE
186,Royal Oak School,9100 Royal Birch Blvd NW
187,Lord Beaverbrook High School,9019 Fairmount Dr SE
188,Somerset School,150 Somerset Manor SW


## Are there any schools that have grades listed as "Unknown" that you could update based on the enrolment table? Get the names of these schools. Which ones could you not update with the enrolment table? (4 marks)

In [127]:
query8 = '''
SELECT NAME
FROM schools
WHERE GRADES = 'Unknown'
'''
result_df8 = pd.read_sql_query(query8, engine)
result_df8

,NAME
0,CLS Alternative Site
1,CLS Lord Shaughnessy Writing Centre
2,Little Angels
3,Alberta Chung Wah School
4,CLS James Fowler Writing Centre
5,Vista Virtual School South
6,Project Trust
7,Maria Montessori School - cSPACE King Edward L...
8,Renfrew Educational Services - Park Place Centre
9,Filipino Language and Cultural School of Calgary


In [128]:
## Common school names between schools with unknown grades and scenr table (These are the schools that could be updated)

query9 = '''
SELECT DISTINCT s.NAME AS Common_School_Names
FROM schools s
INNER JOIN scenr sc ON s.NAME = sc.`School Name`
WHERE s.GRADES = 'Unknown';
'''
result_df9 = pd.read_sql_query(query9, engine)
result_df9

,Common_School_Names
0,Juno Beach Academy of Canadian Studies
1,W.H.Cushing Workplace School
2,AADAC Youth Services
3,Project Trust
4,Maria Montessori School - cSPACE King Edward L...
5,Calgary International Academy ECS


In [129]:
## School names from schools with unknown grades not found in scenr table
query10 = '''
SELECT DISTINCT s.NAME AS Left_Out_School_Names
FROM schools s
LEFT JOIN scenr sc ON s.NAME = sc.`School Name`
WHERE s.GRADES = 'Unknown' AND sc.`School Name` IS NULL;
'''
result_df10 = pd.read_sql_query(query10, engine)
result_df10


,Left_Out_School_Names
0,CLS Alternative Site
1,CLS Lord Shaughnessy Writing Centre
2,Little Angels
3,Alberta Chung Wah School
4,CLS James Fowler Writing Centre
5,Vista Virtual School South
6,Renfrew Educational Services - Park Place Centre
7,Filipino Language and Cultural School of Calgary
8,Canadian Montessori School West
9,CLS Lord Beaverbrook Writing Centre


### Yes, there are schools with grades listed as 'unknown'. We can likely update these grades by cross-referencing the enrollment tables to determine which grades are present and then assigning appropriate grade levels to the schools table. Essentially, the schools with unknown grades that cannot be updated are those where the school names are not found in the school enrollment table. (SHOWN ABOVE)

## Where there is ECS enrolment at a private school, include the address, school name, and whether the school is an elementary school or secondary school. (2 marks)

### Yes, there were ECS enrollments at private schools. Please refer to the filtered table below for details

In [130]:

query11 = '''

SELECT s.NAME AS School_Name,
       s.ADDRESS_AB AS School_Address,
       CASE
           WHEN s.ELEM = 'Y' THEN 'Elementary School'
           WHEN s.JUNIOR_H = 'Y' THEN 'Secondary School'
           WHEN s.SENIOR_H = 'Y' THEN 'Secondary School'
           ELSE 'Unknown'
       END AS School_Type,
       sc.`School Authority Category` AS School_Authority_Category,
       CASE
           WHEN sc.ECS > 0 THEN 'Yes'
           ELSE 'No'
       END AS ECS_Enrollments
FROM schools s
JOIN scenr sc ON s.NAME = sc.`School Name`
WHERE sc.`School Authority Category` = 'Private';

'''
result_df11 = pd.read_sql_query(query11, engine)
result_df11


,School_Name,School_Address,School_Type,School_Authority_Category,ECS_Enrollments
0,Montessori School of Calgary,2201 Cliff Street S.W.,Elementary School,Private,Yes
1,Calgary French & International School,700 - 77 Street S.W.,Elementary School,Private,Yes
2,Calgary Academy Collegiate,1677 - 93rd Street S.W.,Elementary School,Private,No
3,Aurora Learning Calgary,#137 5305 McCall Way NE,Elementary School,Private,No
4,Maria Montessori School,Chief Crowfoot School Building \n2634 12 Ave NW,Elementary School,Private,Yes
...,...,...,...,...,...
136,Columbia College,802 Manning Road N.E.,Secondary School,Private,No
137,Columbia College,802 Manning Road N.E.,Secondary School,Private,No
138,Calgary Quest School,3405 Spruce Drive S.W. \nc/o Spruce Cliff Elem...,Elementary School,Private,No
139,Calgary French & International School,700 - 77 Street S.W.,Elementary School,Private,Yes


## Report the name and addresses of the schools that have a ECS program and enrolment number for each year from 2013-2022. What is the breakdown of school types (grades in the School dataset) that have ECS enrolment in the most recent year of the enrolment dataset? (4 marks)

In [131]:
query12 = '''

SELECT s.NAME AS School_Name,
       s.ADDRESS_AB AS School_Address,
       s.POSTSECOND AS School_Type,
       CASE
           WHEN s.ELEM = 'Y' THEN 'Elementary School'
           WHEN s.JUNIOR_H = 'Y' THEN 'Secondary School'
           WHEN s.SENIOR_H = 'Y' THEN 'Secondary School'
           ELSE 'Unknown'
       END AS School_Type_Description,
       sc.`School Year`,
       sc.ECS AS ECS_Enrollments
FROM schools s
JOIN scenr sc ON s.NAME = sc.`School Name`
WHERE sc.ECS > 0
  AND sc.`School Year` BETWEEN 2013 AND 2022
ORDER BY s.NAME, sc.`School Year`;


'''

result_df12 = pd.read_sql_query(query12, engine)
result_df12


,School_Name,School_Address,School_Type,School_Type_Description,School Year,ECS_Enrollments
0,Abbeydale School,320 Abergale Dr NE,N,Elementary School,2013_2014,66.0
1,Abbeydale School,320 Abergale Dr NE,N,Elementary School,2014_2015,53.0
2,Abbeydale School,320 Abergale Dr NE,N,Elementary School,2015_2016,60.0
3,Abbeydale School,320 Abergale Dr NE,N,Elementary School,2016_2017,44.0
4,Abbeydale School,320 Abergale Dr NE,N,Elementary School,2017_2018,46.0
...,...,...,...,...,...,...
2363,Woodlands School,88 Woodgreen Dr SW,N,Elementary School,2018_2019,31.0
2364,Woodlands School,88 Woodgreen Dr SW,N,Elementary School,2019-2020,33.0
2365,Woodlands School,88 Woodgreen Dr SW,N,Elementary School,2020-2021,32.0
2366,Woodlands School,88 Woodgreen Dr SW,N,Elementary School,2021-2022,38.0


## Part C: Evaluating your results  (6 marks)

**Question 1 (2 marks)**

## One thing you may have had consider is your selection of columns from both datasets to use as a key for any joins you performed. Discuss your reasoning behind your choice of key. If you don't think you had a reason in particular, then name another pair of colums which could have been used to execute your joins.

The choice of join key in the SQL query was based on the following considerations:

Uniqueness: The School Name and NAME columns were selected as they are expected to uniquely identify each school within their respective datasets.

Relevance: Both columns represent the name of the school, making them fundamental pieces of information present in both datasets.

Consistency: The selected columns have consistent data types and values across both tables, ensuring a successful join operation.

An alternative pair of columns that could have been used are School Code or School ID, provided they serve as unique identifiers for each chool.

Overall, the choice of join key prioritized uniqueness, relevance, and consistency to facilitate an effective join operation between the datasets.

**Question 3 (4 marks)**

It is possible to download the School dataset as a GeoJSON file rather than as a CSV. Imagine that we have loaded this GeoJSON files into MongoDB instead of a relational database.

Do you think it is more easy or difficult to retrieve certain information requested from a table, or from a MongoDB collection? Explain why or why not. 

Consider the result of a join that you had to perform using the datasets. Is there any reason you would store the tables or result in MongoDB as opposed to a relational database? Why or why not?

Yes, it would be possible. We could store data relevant to each school in separate JSON objects, each of which has been added to a document in the same collection in MongoDB. Overall, MongoDB can be faster than SQL if the data is appropriately indexed. Additionally, MongoDB is well-suited for handling hierarchical data. The more nested the data becomes, the more I would lean towards using MongoDB over SQL. Personally, I prefer SQL simply because I have more practice with it, but I believe MongoDB could become easier with more practice.

## Part E: Reflection (4 marks)

In a brief paragraph for each (no more than 500 words total), answer the following:


1) Identify a skill or concept which you are now more knowledgable or comfortable with now, compared to at the start of DATA 604. 


2) What best helped you to learn this skill or concept? Was it something covered in class, part of an assignment or project, or another resource?


3) Based upon what you have learned, where do you see an opportunity to continue to develop your understanding of this skill? 




Throughout DATA 604, my proficiency in SQL querying and understanding database structures has significantly improved. At the start of the course, I had little knowledge about SQL, but now I feel comfortable writing queries and comprehending database concepts. The assignments in DATA 604, particularly assignment three, played a crucial role in my learning. Assignment three focused on SQL joins, which was initially challenging for me. However, through hands-on practice I gained a deeper understanding of joins and their applications. Looking ahead, I aim to continue developing my SQL skills through real-world application. The datasets used in assignment three are coincidentally the same ones used in our group's final project. Leveraging the techniques learned in assignment three, I feel more confident about integrating multiple datasets and extracting insights for our final project. Additionally, exploring advanced SQL functionalities like subqueries and window functions will further enhance my proficiency in data analysis. Continued practice, exposure to diverse datasets, and collaborative projects will be essential for advancing my understanding of SQL and database management.

## References

Both datasets used in this Assignment are licensed under the Open Government License - City of Calgary.

School Enrolment Data [online], May 12, 2023. Open Data (City of Calgary). Available from: https://data.calgary.ca/Demographics/School-Enrolment-Data/9qye-mibh/about_data [Accessed 24 Mar 2024].

Schools [online], March 1, 2024. Open Data (City of Calgary). Available from: https://data.calgary.ca/Services-and-Amenities/Schools/fd9t-tdn2/about_data [Accessed 24 Mar 2024].
